# Initialization

In [1]:
# Google Earth Engine packages
import ee
import geemap
import geemap.colormaps as cm

import xarray as xr
import numpy as np


# function to load nc file into GEE
def netcdf_to_ee(filename):
    ds = xr.open_dataset(filename)
    data = ds['z']

    lon_data = np.round(data['lon'], 3)
    lat_data = np.round(data['lat'], 3)

    dim_lon = np.unique(np.ediff1d(lon_data).round(3))
    dim_lat = np.unique(np.ediff1d(lat_data).round(3))

    if (len(dim_lon) != 1) or (len(dim_lat) != 1):
        print("The netCDF file is not a regular longitude/latitude grid")

    data_np = np.array(data)
    data_np = np.transpose(data_np)

    # Figure out if we need to roll the data or not
    # (see https://github.com/giswqs/geemap/issues/285#issuecomment-791385176)
    if np.max(lon_data) > 180:
        data_np = np.roll(data_np, 180, axis=0)
        west_lon = lon_data[0] - 180
    else:
        west_lon = lon_data[0]

    transform = [dim_lon[0], 0, float(west_lon) - dim_lon[0]/2, 0, dim_lat[0], float(lat_data[0]) - dim_lat[0]/2]

    image = geemap.numpy_to_ee(
        data_np, "EPSG:4326", transform=transform, band_names='z'
    )
    return image, data_np, transform

In [2]:
# initialize GEE at the beginning of session
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()         # authenticate when using GEE for the first time
    ee.Initialize()
    
Map = geemap.Map()

***

# ERA5L Geopotential

### Load input data

Load ERA5L file for geopotential and add to Map

In [3]:
path = 'input/'
file = 'ERA5_land_Z_geopotential_HMA.nc'
filename = path+file

In [4]:
image, data_np, transform = netcdf_to_ee(filename)

In [5]:
# add image as layer
vis_params =  {'min': int(data_np.min()), 'max': int(data_np.max()), 'palette': cm.palettes.terrain, 'opacity': 0.8}
Map.addLayer(image, vis_params, "ERA5L geopotential")

Load catchment and add to map

In [6]:
import geopandas as gpd

output_folder = 'output/'
catchment_new = gpd.read_file('output/catchment_data.gpkg', layer='catchment_new')

catchment = geemap.geopandas_to_ee(catchment_new)
Map.addLayer(catchment, {'color': 'darkgrey'}, "Catchment")
Map.centerObject(catchment, zoom=9)

Show Map

In [7]:
Map

Map(center=[42.183061016411926, 78.19022020297338], controls=(WidgetControl(options=['position', 'transparent_…

### Calculate weighted avarage geopotential and covert to elevation

In [8]:
# execute reducer
dict = image.reduceRegion(ee.Reducer.mean(),
                          geometry=catchment,
                          crs='EPSG:4326',
                          crsTransform=transform)

# get mean value and print
mean_val = dict.getInfo()['z']
print(f'geopotential mean: {mean_val:.2f}, elevation: {mean_val / 9.80665:.2f}')

geopotential mean: 32731.77, elevation: 3337.71


***

### ERA5L Temperature and Precipitation Time Series

In [9]:
collection = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_RAW')\
    .select('temperature_2m','total_precipitation_sum')\
    .filterDate('1982-01-01', '1982-12-31')\
    .filterBounds(catchment)

In [10]:
def setProperty(image):
    dict = image.reduceRegion(ee.Reducer.mean(), catchment)
    return image.set(dict)


withMean = collection.map(setProperty)

In [11]:
import pandas as pd
import datetime

df = pd.DataFrame()
df['ts'] = withMean.aggregate_array('system:time_start').getInfo()
df['temp'] = withMean.aggregate_array('temperature_2m').getInfo()
df['temp_corr'] = df['temp'] + 4.68
df['temp_corr_c'] = df['temp'] - 273.15
df['prec'] = withMean.aggregate_array('total_precipitation_sum').getInfo()
df['prec'] = df['prec'] * 1000
df['dt'] = df['ts'].apply(lambda x: datetime.datetime.fromtimestamp(x / 1000))

In [12]:
df.to_csv(output_folder + 'ERA5L.csv',header=True,index=False)

In [13]:
display(df)

,ts,temp,temp_corr,temp_corr_c,prec,dt
0,378691200000,254.942000,259.622000,-18.208000,0.060925,1982-01-01 01:00:00
1,378777600000,253.960229,258.640229,-19.189771,0.013262,1982-01-02 01:00:00
2,378864000000,254.730269,259.410269,-18.419731,0.111354,1982-01-03 01:00:00
3,378950400000,257.254092,261.934092,-15.895908,0.504703,1982-01-04 01:00:00
4,379036800000,260.315701,264.995701,-12.834299,1.016434,1982-01-05 01:00:00
...,...,...,...,...,...,...
359,409708800000,261.351846,266.031846,-11.798154,0.000852,1982-12-26 01:00:00
360,409795200000,262.988467,267.668467,-10.161533,0.001737,1982-12-27 01:00:00
361,409881600000,260.924753,265.604753,-12.225247,0.124551,1982-12-28 01:00:00
362,409968000000,259.694680,264.374680,-13.455320,7.109920,1982-12-29 01:00:00
